In [1]:
#coding: utf-8
import numpy as np
import pandas as pd
import re
import jieba
import os
import time
import jieba  
import numpy as np
from progressbar import *
start1 =time.clock()

C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # This is added back by InteractiveShellApp.init_path()


In [2]:
def obatin_news_from_csv(path_csv):
    old_text=pd.read_csv(path_csv)
    drop_index_na=[]
    for i in range(old_text.shape[0]):
        j=old_text['微博正文'][i]
        if ("抱歉" in j or len(j)<40):
            drop_index_na.append(i)
    text=old_text.drop(drop_index_na)
    text=text.reset_index(drop=True)
    return text

import re
def find_title(text):
    text["标题"]=[-99 for i in range(text.shape[0])]
    text["微博正文（去掉标题）"]=[-99 for i in range(text.shape[0])]
    notitle_index=[]
    title_p=re.compile(r".*\【.*\】")
    progress = ProgressBar()
    for i in progress(range(text.shape[0])):
        title_=re.match(title_p,text["微博正文"][i])
        if title_:
            text["标题"][i]=title_[0]
            text["微博正文（去掉标题）"][i]=re.sub(r".*\【.*\】","",text["微博正文"][i])
        else:
            notitle_index.append(i)
        time.sleep(0.1)
    text=text.drop(notitle_index)
    text=text.reset_index(drop=True)
    return text

In [3]:

jieba.load_userdict(r".\自定义词典.txt")
stopwords = [line.strip() for line in open(r".\停用词库.txt",encoding='UTF-8').readlines()]

def clean_text_content(text_content,cut_all=False):
    text_content = re.sub(r"。", ".", text_content)
    text_content = re.sub(r"，", ",", text_content)
    text_content = re.sub(r"“", "'", text_content)
    text_content = re.sub(r"”", "'", text_content)
    text_content = re.sub(r"…", ".", text_content)
    text_content = re.sub(r"@", "@", text_content)
    text_content = re.sub(r" ", " ", text_content)
    text_content = re.sub(r"！", "!", text_content)
    text_content = re.sub(r"？", "?", text_content)
    text_content = re.sub(r"：", ":", text_content)
    text_content = re.sub(r"）", ")", text_content)
    text_content = re.sub(r"（", "(", text_content)
    text_content = re.sub(r"(\d+年)*(\d+月)*(\d+[日])", "", text_content) #日期
    text_content = re.sub(r"\d+[年月日天号人名时例名省市区县院]", "", text_content) 
    text_content = re.sub(r"[第]*[零一二三四五六七八九百千万]+[年月日天号人名时例名省市区县院名例周月年]*", "", text_content) 
    text_content = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text_content) #时间
    text_content = re.sub(r"^[-+]?[0-9]+(\.)?[0-9]*$", "", text_content) #数字
    text_content = re.sub(r"@\S*\:+\s*", " ", text_content) #@小央视频
    text_content = re.sub(r"\[.+\]", "", text_content) #[组图共2张]和[加油]
    text_content = re.sub(r"\(.*?\)", "", text_content) #（环球网）
    
    text_content = re.sub(r"\W+\w*(视频)\s+", "", text_content) #小央视频的秒拍视频
    text_content = re.sub(r"#", "", text_content) ##
    text_content = re.sub(r"(http|https)(://t.cn/)[a-zA-Z0-9]+", "", text_content) #网址(微博上的连接都是http://t.cn/.....形式)
    text_content = re.sub(r"转发理由:", "", text_content) 
    text_content = re.sub(r"转发内容:", "", text_content) 
    text_content = re.sub(r"原始用户:.*", "", text_content) 
    
    word_list=jieba.lcut(text_content,cut_all=cut_all)
    word_list_len=len(word_list)
    i=0
    while i<word_list_len:
        j=word_list[i]
        if j in stopwords or not('\u4e00'<=j<='\u9fff') or len(j)<2:
            word_list.pop(i)
            word_list_len=word_list_len-1
        elif (j=="中方" or j=="我国"):
            word_list[i]="中国"
        elif j=="美方":
            word_list[i]="美国"
        elif j=="贸易战":
            word_list[i]="贸易" 
        else:
            i=i+1
    if " " in word_list:
        word_list.remove(" ")
    return word_list#text_content 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Public\Documents\Wondershare\CreatorTemp\jieba.cache
Loading model cost 0.965 seconds.
Prefix dict has been built successfully.


In [4]:
old_text=obatin_news_from_csv("./1974576991.csv")
text_column=["微博正文","发布时间"]

#把2020年的都去掉
len_old_text=old_text.shape[0]
drop_index=[]
old_pattern=re.compile(r"2020(\-)[0-9]+(\-)[0-9]+")
for i in range(0,len_old_text):
    if old_pattern.match(old_text["发布时间"][i]):
        drop_index.append(i)
text=old_text.drop(drop_index)
text=text.reset_index(drop=True)
text.head(2)

text=find_title(text)
text_shape_old=text.shape[0]
print(text_shape_old)
text.head()

N/A% (0 of 20518) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dell-pc\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100% (20518 of 205

17682


,微博id,微博正文,原始图片url,被转发微博原始图片url,是否为原创微博,微博视频url,发布位置,发布时间,发布工具,点赞数,转发数,评论数,标题,微博正文（去掉标题）
0,Inng7t7Qo,【如何获得稳定的8小时睡眠？】如今随着新的信息技术的发展，比如笔记本电脑，手机，游戏成了影响...,无,无,True,无,无,2019-12-31 23:50,微博 weibo.com,506,331,34,【如何获得稳定的8小时睡眠？】,如今随着新的信息技术的发展，比如笔记本电脑，手机，游戏成了影响着人们的睡眠时间的主要原因，7...
1,Inn80A0nV,【这妈妈不太省心！怕被抓拍#女子拿孩子的画遮号牌#】近日，四川自贡市一女子违停在路边，由于担...,无,无,True,http://miaopai.video.weibocdn.com/000mhB9Blx07...,无,2019-12-31 23:30,微博 weibo.com,368,24,79,【这妈妈不太省心！怕被抓拍#女子拿孩子的画遮号牌#】,近日，四川自贡市一女子违停在路边，由于担心被监控抓拍，竟用孩子的画遮挡号牌。孩子制止其母无果...
2,InmZSs4Wv,【#致敬2019那些温暖瞬间#，愿2020更美好】回顾2019年，发生在齐鲁大地上，值得记忆...,无,无,True,无,无,2019-12-31 23:10,微博 weibo.com,317,69,19,【#致敬2019那些温暖瞬间#，愿2020更美好】,回顾2019年，发生在齐鲁大地上，值得记忆的暖心瞬间：10岁男孩2月增重12斤捐造干救父；夫...
3,InmRLvKiu,【浪漫！#85岁爷爷敬老院娶82岁奶奶#：上海话“搭讪”，开音乐会庆祝】 陕西西安一敬老院，...,无,无,True,无,无,2019-12-31 22:50,微博 weibo.com,1273,36,64,【浪漫！#85岁爷爷敬老院娶82岁奶奶#：上海话“搭讪”，开音乐会庆祝】,陕西西安一敬老院，85岁的陈爷爷迎娶82岁的沈奶奶。沈奶奶爱在大厅里弹钢琴，陈爷爷常坐在后...
4,InmJEgLDC,【瑞典环保少女：不想和特朗普说话，不想浪费时间】12月30日，瑞典环保少女通贝里接受BBC《...,无,无,True,无,无,2019-12-31 22:30,微博 weibo.com,2867,71,222,【瑞典环保少女：不想和特朗普说话，不想浪费时间】,12月30日，瑞典环保少女通贝里接受BBC《今日》电台采访时，被问到如果有机会跟特朗普面对面...


In [5]:
start =time.clock()

len_text=text.shape[0]

text["微博正文(无标题切词后)"]=[-99 for i in range(len_text)]
text["标题(切词后)"]=[-99 for i in range(len_text)]

drop_na=[]

progress = ProgressBar()

for u in progress(range(0,len_text)):
    cleaned_content=clean_text_content(text["微博正文"][u])
    text["微博正文(无标题切词后)"][u]=" ".join(cleaned_content)

    cleaned_title=clean_text_content(text["标题"][u],True)
    text["标题(切词后)"][u]=" ".join(cleaned_title)

    if pd.isnull(text.loc[u]).any():
        drop_na.append(u)
        print(u) 
    time.sleep(0.1)

text=text.drop(drop_na)
text=text.reset_index(drop=True)   

text["微博正文(有标题切词后)"]=[-99 for i in range(len_text)]
progress = ProgressBar()

for u in progress(range(0,len_text)):
    text["微博正文(有标题切词后)"][u]=text["标题(切词后)"][u]+" "+text["微博正文(无标题切词后)"][u]

end = time.clock()

print('Running time: %s Seconds'%(end-start))

C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
N/A% (0 of 17682) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100% (17682 of 17682) |##################| Elapsed Time: 0:49:02 Time:  0:4

Running time: 3328.6935436999997 Seconds


C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [6]:
print("text_shape_old",text_shape_old)
print("text_shape_new",text.shape[0])
text.head()

text_shape_old 17682
text_shape_new 17682


,微博id,微博正文,原始图片url,被转发微博原始图片url,是否为原创微博,微博视频url,发布位置,发布时间,发布工具,点赞数,转发数,评论数,标题,微博正文（去掉标题）,微博正文(无标题切词后),标题(切词后),微博正文(有标题切词后)
0,Inng7t7Qo,【如何获得稳定的8小时睡眠？】如今随着新的信息技术的发展，比如笔记本电脑，手机，游戏成了影响...,无,无,True,无,无,2019-12-31 23:50,微博 weibo.com,506,331,34,【如何获得稳定的8小时睡眠？】,如今随着新的信息技术的发展，比如笔记本电脑，手机，游戏成了影响着人们的睡眠时间的主要原因，7...,获得 稳定 小时 睡眠 信息技术 发展 笔记本电脑 手机 游戏 影响 人们 睡眠 主要 原因...,获得 稳定 小时 睡眠,获得 稳定 小时 睡眠 获得 稳定 小时 睡眠 信息技术 发展 笔记本电脑 手机 游戏 影响...
1,Inn80A0nV,【这妈妈不太省心！怕被抓拍#女子拿孩子的画遮号牌#】近日，四川自贡市一女子违停在路边，由于担...,无,无,True,http://miaopai.video.weibocdn.com/000mhB9Blx07...,无,2019-12-31 23:30,微博 weibo.com,368,24,79,【这妈妈不太省心！怕被抓拍#女子拿孩子的画遮号牌#】,近日，四川自贡市一女子违停在路边，由于担心被监控抓拍，竟用孩子的画遮挡号牌。孩子制止其母无果...,妈妈 不太 省心 抓拍 孩子 号牌 自贡市 违停 路边 担心 监控 抓拍 孩子 遮挡 号牌 ...,妈妈 省心 抓拍 孩子 号牌,妈妈 省心 抓拍 孩子 号牌 妈妈 不太 省心 抓拍 孩子 号牌 自贡市 违停 路边 担心 ...
2,InmZSs4Wv,【#致敬2019那些温暖瞬间#，愿2020更美好】回顾2019年，发生在齐鲁大地上，值得记忆...,无,无,True,无,无,2019-12-31 23:10,微博 weibo.com,317,69,19,【#致敬2019那些温暖瞬间#，愿2020更美好】,回顾2019年，发生在齐鲁大地上，值得记忆的暖心瞬间：10岁男孩2月增重12斤捐造干救父；夫...,致敬 温暖 瞬间 美好 回顾 齐鲁大地 值得 记忆 瞬间 男孩 增重 捐造 干救父 夫妻 雾...,致敬 温暖 瞬间 美好,致敬 温暖 瞬间 美好 致敬 温暖 瞬间 美好 回顾 齐鲁大地 值得 记忆 瞬间 男孩 增重...
3,InmRLvKiu,【浪漫！#85岁爷爷敬老院娶82岁奶奶#：上海话“搭讪”，开音乐会庆祝】 陕西西安一敬老院，...,无,无,True,无,无,2019-12-31 22:50,微博 weibo.com,1273,36,64,【浪漫！#85岁爷爷敬老院娶82岁奶奶#：上海话“搭讪”，开音乐会庆祝】,陕西西安一敬老院，85岁的陈爷爷迎娶82岁的沈奶奶。沈奶奶爱在大厅里弹钢琴，陈爷爷常坐在后...,浪漫 爷爷 敬老院 奶奶 上海 搭讪 音乐会 庆祝 陕西 西安 敬老院 爷爷 迎娶 奶奶 奶...,浪漫 爷爷 敬老 敬老院 奶奶 上海 搭讪 音乐 音乐会 会庆 庆祝,浪漫 爷爷 敬老 敬老院 奶奶 上海 搭讪 音乐 音乐会 会庆 庆祝 浪漫 爷爷 敬老院 奶...
4,InmJEgLDC,【瑞典环保少女：不想和特朗普说话，不想浪费时间】12月30日，瑞典环保少女通贝里接受BBC《...,无,无,True,无,无,2019-12-31 22:30,微博 weibo.com,2867,71,222,【瑞典环保少女：不想和特朗普说话，不想浪费时间】,12月30日，瑞典环保少女通贝里接受BBC《今日》电台采访时，被问到如果有机会跟特朗普面对面...,瑞典 环保 少女 不想 特朗普 说话 不想 浪费时间 瑞典 环保 少女 贝里 接受 今日 电...,瑞典 环保 少女 不想 特朗普 说话 不想 浪费 浪费时间 费时 费时间,瑞典 环保 少女 不想 特朗普 说话 不想 浪费 浪费时间 费时 费时间 瑞典 环保 少女 ...


In [14]:
text_new=text[["发布时间","微博正文","标题","微博正文（去掉标题）","微博正文(无标题切词后)","标题(切词后)","微博正文(有标题切词后)"]]
text_new.head()

,发布时间,微博正文,标题,微博正文（去掉标题）,微博正文(无标题切词后),标题(切词后),微博正文(有标题切词后)
0,2019-12-31 23:50,【如何获得稳定的8小时睡眠？】如今随着新的信息技术的发展，比如笔记本电脑，手机，游戏成了影响...,【如何获得稳定的8小时睡眠？】,如今随着新的信息技术的发展，比如笔记本电脑，手机，游戏成了影响着人们的睡眠时间的主要原因，7...,获得 稳定 小时 睡眠 信息技术 发展 笔记本电脑 手机 游戏 影响 人们 睡眠 主要 原因...,获得 稳定 小时 睡眠,获得 稳定 小时 睡眠 获得 稳定 小时 睡眠 信息技术 发展 笔记本电脑 手机 游戏 影响...
1,2019-12-31 23:30,【这妈妈不太省心！怕被抓拍#女子拿孩子的画遮号牌#】近日，四川自贡市一女子违停在路边，由于担...,【这妈妈不太省心！怕被抓拍#女子拿孩子的画遮号牌#】,近日，四川自贡市一女子违停在路边，由于担心被监控抓拍，竟用孩子的画遮挡号牌。孩子制止其母无果...,妈妈 不太 省心 抓拍 孩子 号牌 自贡市 违停 路边 担心 监控 抓拍 孩子 遮挡 号牌 ...,妈妈 省心 抓拍 孩子 号牌,妈妈 省心 抓拍 孩子 号牌 妈妈 不太 省心 抓拍 孩子 号牌 自贡市 违停 路边 担心 ...
2,2019-12-31 23:10,【#致敬2019那些温暖瞬间#，愿2020更美好】回顾2019年，发生在齐鲁大地上，值得记忆...,【#致敬2019那些温暖瞬间#，愿2020更美好】,回顾2019年，发生在齐鲁大地上，值得记忆的暖心瞬间：10岁男孩2月增重12斤捐造干救父；夫...,致敬 温暖 瞬间 美好 回顾 齐鲁大地 值得 记忆 瞬间 男孩 增重 捐造 干救父 夫妻 雾...,致敬 温暖 瞬间 美好,致敬 温暖 瞬间 美好 致敬 温暖 瞬间 美好 回顾 齐鲁大地 值得 记忆 瞬间 男孩 增重...
3,2019-12-31 22:50,【浪漫！#85岁爷爷敬老院娶82岁奶奶#：上海话“搭讪”，开音乐会庆祝】 陕西西安一敬老院，...,【浪漫！#85岁爷爷敬老院娶82岁奶奶#：上海话“搭讪”，开音乐会庆祝】,陕西西安一敬老院，85岁的陈爷爷迎娶82岁的沈奶奶。沈奶奶爱在大厅里弹钢琴，陈爷爷常坐在后...,浪漫 爷爷 敬老院 奶奶 上海 搭讪 音乐会 庆祝 陕西 西安 敬老院 爷爷 迎娶 奶奶 奶...,浪漫 爷爷 敬老 敬老院 奶奶 上海 搭讪 音乐 音乐会 会庆 庆祝,浪漫 爷爷 敬老 敬老院 奶奶 上海 搭讪 音乐 音乐会 会庆 庆祝 浪漫 爷爷 敬老院 奶...
4,2019-12-31 22:30,【瑞典环保少女：不想和特朗普说话，不想浪费时间】12月30日，瑞典环保少女通贝里接受BBC《...,【瑞典环保少女：不想和特朗普说话，不想浪费时间】,12月30日，瑞典环保少女通贝里接受BBC《今日》电台采访时，被问到如果有机会跟特朗普面对面...,瑞典 环保 少女 不想 特朗普 说话 不想 浪费时间 瑞典 环保 少女 贝里 接受 今日 电...,瑞典 环保 少女 不想 特朗普 说话 不想 浪费 浪费时间 费时 费时间,瑞典 环保 少女 不想 特朗普 说话 不想 浪费 浪费时间 费时 费时间 瑞典 环保 少女 ...


In [15]:
outputpath='./huanqiu_news_with_title_2.csv'
text_new.to_csv(outputpath,sep=',',index=False,header=True)

In [ ]:
temp_text=text["微博正文(有标题切词后)"].split(" ")
all_words=[str(i) for k in temp_text for i in k]
all_words_space=" ".join(('%s' %id for id in all_words))
#all_words_space[:200]

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
wordcloud1 = WordCloud(
       font_path="C:/Windows/Fonts/simfang.ttf",
       background_color="white",
       width=800,
       height=660,
       max_font_size=200).generate(all_words_space)
plt.imshow(wordcloud1.recolor(), interpolation="bilinear")
plt.axis("off")
wordcloud1.to_file('./wordcloud1.jpg')
plt.show()

In [ ]:
import jieba.analyse
tag=jieba.analyse.extract_tags(all_words_space, topK=100, withWeight=True, allowPOS=())
wc_tfidf ={}
for i in tag:
    wc_tfidf[i[0]]=i[1]
    
wordcloud2 = WordCloud(
       font_path="C:/Windows/Fonts/simfang.ttf",
       background_color="white",
       width=800,
       height=660,
       max_font_size=200).generate_from_frequencies(wc_tfidf)
plt.imshow(wordcloud2.recolor(), interpolation="bilinear")
plt.axis("off")
wordcloud2.to_file('./result2.jpg')
plt.show()

In [ ]:
import re
time_all={}
for i in range(1,13):
    month_text=0
    if len(str(i))==1:
        mon="0"+str(i)
    else:
        mon=str(i)
    month_re='2019-'+mon
    
    time_list=text["发布时间"]
    for j in time_list:
        if month_re in j:
            month_text=month_text+1
    time_all[mon]=month_text
print(time_all)

import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
month_re_list=[]
for i in range(1,13):
    if len(str(i))==1:
        mon="0"+str(i)
    else:
        mon=str(i)
    month_re='2019-'+mon
    month_re_list.append(month_re)
#print("month_re_list",month_re_list)
new_ticks =month_re_list
x = [int(i[0]) for i in time_all.items()]
y = [int(i[1]) for i in time_all.items()]
print("x",x)
print("y",y)
print("month_re_list",month_re_list)
peint("sum(y)",sum(y))

plt.figure(figsize=(10,8),dpi=50)
plt.ylim(0,2000)
plt.xticks(rotation = 90)
plt.bar(month_re_list ,y,width=0.8)
for m,n in zip(x ,y):
    plt.text(m-0.7, n+1, '%.0f' % n, ha='right', va= 'bottom',fontsize=11)